In [ ]:
import nltk
from sklearn_crfsuite import CRF
from nltk.corpus.reader import ConllChunkCorpusReader
import re
from sklearn.model_selection import train_test_split
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics._classification import classification_report
from sklearn.naive_bayes import MultinomialNB
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [ ]:
sentenceNtag = []

with open("D:/PythonProject/CRF_tokenize/IOB_data/train1.txt", encoding='utf8') as file:
    for line in file:
        cnt = 0
        word_buff = ""
        tag_buff = ""
        
        while line[cnt].isspace() != True:
            word_buff += line[cnt]
            cnt += 1
        
        cnt = len(line) - 2 
        while line[cnt].isspace() != True: 
            tag_buff += line[cnt]
            cnt -= 1
        tag_buff = tag_buff[::-1]
        
        sentenceNtag.append((word_buff, tag_buff))


In [ ]:
sentenceNtag[0][0]

In [ ]:
sentences = []

temp = []
for i in range(len(sentenceNtag)):
    if sentenceNtag[i][0] != '.':
        temp.append(sentenceNtag[i])
    else:
        sentences.append(temp)
        temp = []

In [ ]:
print(len(sentences))

In [ ]:
def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [ ]:
%%time
X_train = [sent2features(s) for s in sentences]
y_train = [sent2labels(s) for s in sentences]

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=None)


In [ ]:
len(y_train)

In [ ]:
%%time

crf = CRF(algorithm = "lbfgs",
          c1 = 0.1, 
          c2 = 0.1, 
          max_iterations= 100, 
          all_possible_transitions=True)
try:
    crf.fit(X_train, y_train)
except:
    pass

In [ ]:
labels = list(crf.classes_)
labels.remove('')
labels

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

In [ ]:
import pickle

pickle.dump(crf, open('crf_model.pkl', 'wb'))